In [2]:
#import os
import numpy as np
import pandas as pd
from sklearn import tree
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 120)

In [3]:
input_file = "sample-superstore_2023_T3.csv"
df = pd.read_csv(input_file, header = 0)
df.head(10)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,7773,CA-2016-108196,25/11/2016,12/02/2016,Standard Class,CS-12505,Cindy Stewart,Consumer,United States,Lancaster,...,43130,Est,TEC-MA-10000418,Technology,Machines,Cubify CubeX 3D Printer Double Head Print,4499.985,5,0.7,-6599.978
1,684,US-2017-168116,11/04/2017,11/04/2017,Same Day,GT-14635,Grant Thornton,Corporate,United States,Burlington,...,"""27217""",South,TEC-MA-10004125,Technology,Machines,Cubify CubeX 3D Printer Triple Head Print,7999.980,4,0.5,-3839.9904
2,9775,CA-2014-169019,26/07/2014,30/07/2014,Standard Class,LF-17185,Luke Foster,Consumer,United States,San Antonio,...,78207,Central,OFF-BI-10004995,Office Supplies,Binders,GBC DocuBind P400 Electric Binding System,2177.584,8,0.8,-3701.8928
3,3012,CA-2017-134845,17/04/2017,24/04/2017,Standard Class,SR-20425,Sharelle Roach,Home Office,United States,Louisville,...,80027,West,TEC-MA-10000822,Technology,Machines,Lexmark MX611dhe Monochrome Laser Printer,2549.985,5,0.7,-3399.98
4,4992,US-2017-122714,12/07/2017,13/12/2017,Standard Class,HG-14965,Henry Goldwyn,Corporate,United States,Chicago,...,60653,Central,OFF-BI-10001120,Office Supplies,Binders,Ibico EPK-21 Electric Binding System,1889.990,5,0.8,-2929.4845
5,3152,CA-2015-147830,15/12/2015,18/12/2015,First Class,NF-18385,Natalie Fritzler,Consumer,United States,Newark,...,43055,East,TEC-MA-10000418,Technology,Machines,Cubify CubeX 3D Printer Double Head Print,1799.994,Two,0.7,"""-2639.9912"""
6,5311,CA-2017-131254,19/11/2017,21/11/2017,First Class,NC-18415,Nathan Cano,Consumer,United States,Houston,...,77095,Central,OFF-BI-10003527,Office Supplies,Binders,Fellowes PB500 Electric Punch Plastic Comb Binding Machine with Manual Bind,1525.188,6,0.8,-2287.782
7,9640,CA-2015-116638,28/01/2015,NaN,Second Class,JH-15985,Joseph Holt,Consumer,United States,Concord,...,28027,South,FUR-TA-10000198,Frnture,Tables,Chromcraft Bull-Nose Wood Oval Conference Tables & Bases,4297.644,Thirteen,0.4,NaN
8,1200,CA-2016-130946,04/08/2016,04/12/2016,Standard Class,ZC-21910,Zuschuss Carroll,Consumer,United States,Houston,...,77041,Central,OFF-BI-10004995,Office Supplies,Binders,GBC DocuBind P400 Electric Binding System,1088.792,4,0.8,-1850.9464
9,2698,CA-2014-145317,18/03/2014,23/03/2014,Standard Class,SM-20320,Sean Miller,Home Office,NaN,Jacksonville,...,32216,Southh,TEC-MA-10002412,Technology,Machines,Cisco TelePresence System EX90 Videoconferencing Unit,22638.480,6,0.5,-1811.0784


In [4]:
len(df)

9994

In [5]:
print(df.columns)

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')


## Handling missing values in the dataset

### Count missing values in each column

In [6]:
missing_values = df.isnull().sum()
print(missing_values)

Row ID            0
Order ID          1
Order Date        2
Ship Date         3
Ship Mode         4
Customer ID       0
Customer Name     3
Segment           3
Country           4
City              2
State             4
Postal Code       3
Region            3
Product ID        2
Category          2
Sub-Category      4
Product Name      3
Sales             1
Quantity          5
Discount          3
Profit           11
dtype: int64


###  "Imputation with Corresponding Values" for missing values in "Customer Name" column.

To fill in missing 'Customer Name' values using names linked to the unique 'Customer ID' values in the dataset.

In [7]:
#Find 'Customer ID' where 'Customer Name' is missing

missing_customer_name = (df['Customer Name'].isnull()) & (~df['Customer ID'].isnull())
print(df.loc[missing_customer_name, ['Customer ID', 'Customer Name']])

    Customer ID Customer Name
76     SO-20335           NaN
869    DH-13075           NaN
877    BP-11095           NaN


In [8]:
#Compare with specified 'Customer ID' and 'Customer Name' in other rows

customer_ids_missing_name = df.loc[df['Customer Name'].isnull(), 'Customer ID'].unique()

unique_customer_info = df[df['Customer ID'].isin(customer_ids_missing_name)][[
    'Customer ID','Customer Name']].drop_duplicates()
print(unique_customer_info)

     Customer ID   Customer Name
76      SO-20335             NaN
869     DH-13075             NaN
877     BP-11095             NaN
898     DH-13075   Dave Hallsten
1152    BP-11095    Bart Pistole
1628    SO-20335  Sean O'Donnell


In [9]:
# Replace NaN values in 'Customer Name' with "Customer Name" values in corresponding Customer ID"

for index, row in unique_customer_info.iterrows():
    customer_id = row['Customer ID']
    customer_name = df.loc[df['Customer ID'] == customer_id, 'Customer Name'].dropna().iloc[0]
    df.loc[df['Customer ID'] == customer_id, 'Customer Name'] = customer_name
    
specific_customer_name_info = df[df['Customer ID'].isin(['SO-20335', 'DH-13075', 'BP-11095'])][[
    'Customer ID', 'Customer Name']].drop_duplicates()
print(specific_customer_name_info)

    Customer ID   Customer Name
76     SO-20335  Sean O'Donnell
869    DH-13075   Dave Hallsten
877    BP-11095    Bart Pistole


To fill in missing 'Segment' values linked to the unique 'Customer ID' values in the dataset.

In [10]:
#Find 'Customer ID' where 'Segment' is missing

missing_segment = (df['Segment'].isnull()) & (~df['Customer ID'].isnull())
print(df.loc[missing_segment, ['Customer ID', 'Segment',]])

    Customer ID Segment
206    EP-13915     NaN
443    TG-21310     NaN
494    TC-21475     NaN


In [11]:
#Compare with specified 'Customer ID' and 'Segment' in other rows

customer_ids_missing_segment = df.loc[df['Segment'].isnull(), 'Customer ID'].unique()

unique_segment_info = df[df['Customer ID'].isin(customer_ids_missing_segment)][[
    'Customer ID','Segment']].drop_duplicates()
print(unique_segment_info)

    Customer ID      Segment
142    EP-13915     Consumer
163    TG-21310     Consumer
206    EP-13915          NaN
414    TC-21475  Home Office
443    TG-21310          NaN
494    TC-21475          NaN


In [35]:
# Replace NaN values in 'Segment' with "Segment" values in corresponding Customer ID" 

for index, row in unique_segment_info.iterrows():
    customer_id = row['Customer ID']
    segment = df.loc[df['Customer ID'] == customer_id, 'Segment'].dropna().iloc[0]
    df.loc[df['Customer ID'] == customer_id, 'Segment'] = segment
    
specific_segment_info = df[df['Customer ID'].isin(['EP-13915', 'TG-21310', 'TC-21475'])][[
    'Customer ID', 'Segment']]

#print(specific_segment_info)
specific_rows = df.loc[[142, 163, 206, 414, 443, 494], ['Customer ID', 'Segment']]
print(specific_rows)

    Customer ID      Segment
142    EP-13915     Consumer
163    TG-21310     Consumer
206    EP-13915     Consumer
414    TC-21475  Home Office
443    TG-21310     Consumer
494    TC-21475  Home Office


In [ ]:
'''columns1_to_check = ['Row ID', 'Order ID', 'Order Date', 'Ship Date']

for column in columns1_to_check:
    unique_values = df[column].unique().tolist()
    print(f"\nUnique {column}:", unique_values)'''

In [ ]:
columns2_to_check = ['Ship Mode']

for column in columns2_to_check:
    unique_values = df[column].unique().tolist()
    print(f"\nUnique {column}:", unique_values)

In [ ]:
columns_to_check = ['Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
                    'Postal Code', 'Region']

for column in columns_to_check:
    unique_values = df[column].unique().tolist()
    print(f"\nUnique {column}:", unique_values)

In [ ]:
columns_to_check = [Product ID', 'Category', 'Sub-Category','Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']

for column in columns_to_check:
    unique_values = df[column].unique().tolist()
    print(f"\nUnique {column}:", unique_values)

In [ ]:
unique_country = df['Country'].unique()
unique_country = unique_country.tolist()
print("\nUnique Country:", unique_country)

In [ ]:
unique_city = df['City'].unique()
unique_city = unique_city.tolist()
#print("\nUnique city:", unique_city)

In [ ]:
unique_sales = df['Sales'].unique()
unique_sales_list = unique_sales.tolist()
#print("\nUnique Sales:", unique_sales_list)

In [ ]:
unique_quantity = df['Quantity'].unique()
print("\nUnique Quantity:", unique_quantity)

In [ ]:
unique_discount = df['Discount'].unique()
print("\nUnique Discount:", unique_discount)

In [ ]:
unique_discount = df['Discount'].unique()
unique_discount_list = unique_discount.tolist()
print("\nUnique Discount:", unique_discount_list)

## Finding the Outliers

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'df' is your DataFrame and 'Quantity' is the column of interest
qty = df['Quantity']

# Calculate descriptive statistics
mean_qty = qty.mean()
std_qty = qty.std()

# Define a threshold for identifying outliers (e.g., 3 standard deviations from the mean)
outlier_threshold = 3 * std_qty

# Identify outliers
outliers = qty[abs(qty - mean_qty) > outlier_threshold]

# Create a histogram
plt.hist(qty, bins=50, color='blue', alpha=0.7, label='Quantity')
plt.axvline(mean_qty, color='red', linestyle='dashed', linewidth=2, label='Mean')
plt.axvline(mean_qty + outlier_threshold, color='orange', linestyle='dashed', linewidth=2, label='Outlier Threshold')
plt.axvline(mean_qty - outlier_threshold, color='orange', linestyle='dashed', linewidth=2)

# Highlight outliers on the histogram
plt.scatter(outliers, [0] * len(outliers), color='red', label='Outliers')

# Set labels and title
plt.xlabel('Quantity')
plt.ylabel('Frequency')
plt.title('Histogram of Quantity with Outliers')
plt.legend()

# Show the plot
plt.show()


In [ ]:
non_numeric_columns = df.select_dtypes(exclude=['number']).columns
for column in non_numeric_columns:
    unique_values = df[column].unique()
    print(f"{column}: {unique_values}")

In [ ]:
len(df)

In [ ]:
print(df.dtypes)

In [ ]:
#Change 'Quantity' and 'Profit' value type from string to numerical data type

df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Profit'] = pd.to_numeric(df['Profit'], errors='coerce')

In [ ]:
print(df.dtypes)

In [ ]:
#For numrical data, Fill NaN values with mean

mean_sales = df['Sales'].mean()
mean_quantity = df['Quantity'].mean()
mean_discount = df['Discount'].mean()
mean_profit = df['Profit'].mean()

df['Sales'].fillna(mean_sales, inplace=True)
df['Quantity'].fillna(mean_quantity, inplace=True)
df['Discount'].fillna(mean_discount, inplace=True)
df['Profit'].fillna(mean_profit, inplace=True)

In [ ]:
#Round 'Sales', 'Discount', and 'Profit' decimal places

df['Sales'] = df['Sales'].round(1)
df['Quantity'] = df['Quantity'].round(1)
df['Discount'] = df['Discount'].round(2)
df['Profit'] = df['Profit'].round(0)

In [ ]:
df[['Country','City','Customer ID','Product Name','Sales','Quantity','Discount','Profit']].head(10)

In [ ]:
unique_countries = df['Country'].unique()
print("Unique Countries:", unique_countries)

In [ ]:
unique_cities = df['City'].unique()
print("\nUnique Cities:", unique_cities)

In [ ]:
df.head(60)

In [ ]:
# Filter rows where 'Order ID' is 'US-2017-122714'
order_id_row = df[df['Order ID'] == 'US-2017-122714']

# Display the filtered row
print(order_id_row[['Country','City','Product Name','Sales','Quantity','Discount','Profit']])


In [ ]:
descriptive_stats = df.describe()
print(descriptive_stats)

In [ ]:
# Identify and treat outliers in 'Sales' and 'Profit'
# Example: Use z-score to identify and remove outliers
from scipy.stats import zscore
z_scores = zscore(df[['Sales', 'Profit']])
df_no_outliers = df[(z_scores < 3).all(axis=1)]

In [ ]:
# Identify outliers in 'Sales' and 'Profit'
# Apply treatment method, e.g., removing or transforming outliers

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Select numerical variables for normalization and scaling
numerical_vars = ['Sales', 'Quantity', 'Discount', 'Profit']

# Apply Min-Max scaling
scaler = MinMaxScaler()
df[numerical_vars] = scaler.fit_transform(df[numerical_vars])

In [ ]:
# Demonstrate data aggregations or frequency distributions
# Example: Group by 'Category' and calculate the mean of 'Sales' and 'Profit'
group_data = df.groupby('Category')['Sales', 'Profit'].mean()
print(group_data)

In [ ]:
# Identify and handle missing values
missing_values = df.isnull().sum()
# Example: Fill missing values in 'Sales' with the mean
df['Sales'].fillna(df['Sales'].mean(), inplace=True)


In [ ]:
# Describe features that are related and the nature of that relationship
correlation_matrix = df.corr()
print(correlation_matrix)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Example: Univariate analysis and visualization for 'Sales'
plt.figure(figsize=(10, 6))
sns.histplot(df['Sales'], bins=20, kde=True)
plt.title('Distribution of Sales')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.show()
